In [ ]:
from birdy.twitter import UserClient
import json
import networkx as nx
import time

ACCESS_TOKEN = ''
ACCESS_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''


client = UserClient(CONSUMER_KEY,
                    CONSUMER_SECRET,
                    ACCESS_TOKEN,
                    ACCESS_SECRET)

following_ids=client.api.friends.ids.get(screen_name='')
follower_ids=client.api.followers.ids.get(screen_name='')

G=nx.DiGraph()

following=following_ids.data.ids
follower=follower_ids.data.ids

targetFollows = list(set(follower+following))

for id in targetFollows:
    name = client.api.users.show.get(id=id).data.name
    G.add_node(id,label=name)
    
reset_requests=15
resources = ['followers','friends']
rate_limits = client.api.application.rate_limit_status.get(resources=resources).data['resources']['followers']['/followers/ids']['remaining']

while targetFollows:
    id=targetFollows[0]
    print('remaining:',len(targetFollows))
    print('remaining requests:',rate_limits)
    targetFollows.remove(id)
    
    if (rate_limits==0):
        print('raggiunto limite richieste... aspetto 15 minuti')
        time.sleep(61*15)
        rate_limits=reset_requests
        client = UserClient(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_SECRET)
    try:
        following1_ids=client.api.friends.ids.get(id=id).data.ids
        follower1_ids=client.api.followers.ids.get(id=id).data.ids
        rate_limits=rate_limits-1

        ids=following1_ids+follower1_ids
    
        for id1 in ids:  
            if (id1 in targetFollows)and(id1 in following1_ids):
                G.add_edges_from([(id, id1)])
            if (id1 in targetFollows)and(id1 in follower1_ids): 
                G.add_edges_from([(id1, id)])
            
    except Exception as err:
        rate_limits=rate_limits-1
        continue

nx.write_gexf(G, "reteSociale_FINAL.gexf")

Gephi Output: HITS and Modularity Clustering applied

![title](img/SocialNet.png)
